In [1]:
import xl
from utils import *
import tensorflow as tf
from sklearn.cross_validation import train_test_split
import time
import random
import os

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
trainset = sklearn.datasets.load_files(container_path = 'data', encoding = 'UTF-8')
trainset.data, trainset.target = separate_dataset(trainset,1.0)
print (trainset.target_names)
print (len(trainset.data))
print (len(trainset.target))

['negative', 'positive']
10662
10662


In [3]:
concat = ' '.join(trainset.data).split()
vocabulary_size = len(list(set(concat)))
data, count, dictionary, rev_dictionary = build_dataset(concat, vocabulary_size)
print('vocab from size: %d'%(vocabulary_size))
print('Most common words', count[4:10])
print('Sample data', data[:10], [rev_dictionary[i] for i in data[:10]])

vocab from size: 20332
Most common words [('film', 1453), ('movie', 1270), ('one', 727), ('like', 721), ('story', 477), ('much', 386)]
Sample data [7, 15, 89, 10158, 1517, 4373, 10159, 10160, 1785, 55] ['like', 'films', 'almost', 'anthropologically', 'detailed', 'realization', 'early80s', 'suburbia', 'significant', 'without']


In [4]:
GO = dictionary['GO']
PAD = dictionary['PAD']
EOS = dictionary['EOS']
UNK = dictionary['UNK']

In [5]:
learning_rate = 1e-4
batch_size = 32
maxlen = 100
n_layer = 3
d_model = 256
d_embed = 256
n_head = 10
d_head = 50
d_inner = 512

In [6]:
class Model:
    def __init__(self):
        self.X = tf.placeholder(tf.int32, [None, maxlen])
        self.Y = tf.placeholder(tf.int32, [None])
        self.memory = tf.placeholder(tf.float32, [d_model, 0])
        initializer = tf.initializers.random_normal(stddev = 0.1)
        logits, _ = xl.transformer(
            self.X,
            self.memory,
            len(dictionary),
            n_layer,
            d_model,
            d_embed,
            n_head,
            d_head,
            d_inner,
            initializer,
        )
        self.logits = tf.layers.dense(logits[:, -1], 2)
        self.cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits = self.logits, labels = self.Y
            )
        )
        self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(
            self.cost
        )
        correct_pred = tf.equal(
            tf.argmax(self.logits, 1, output_type = tf.int32), self.Y
        )
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [7]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model()
sess.run(tf.global_variables_initializer())

In [8]:
vectors = str_idx(trainset.data,dictionary,maxlen)
train_X, test_X, train_Y, test_Y = train_test_split(vectors, trainset.target,test_size = 0.2)

In [9]:
from tqdm import tqdm
import time

EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 3, 0, 0, 0

while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n' % (EPOCH))
        break

    train_acc, train_loss, test_acc, test_loss = 0, 0, 0, 0
    pbar = tqdm(
        range(0, len(train_X), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        batch_x = train_X[i : min(i + batch_size, train_X.shape[0])]
        batch_y = train_Y[i : min(i + batch_size, train_X.shape[0])]
        batch_memory = np.zeros(([d_model, 0]))
        batch_x_expand = np.expand_dims(batch_x,axis = 1)
        acc, cost, _ = sess.run(
            [model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
                model.memory: batch_memory
            },
        )
        assert not np.isnan(cost)
        train_loss += cost
        train_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)
        
    pbar = tqdm(range(0, len(test_X), batch_size), desc = 'test minibatch loop')
    for i in pbar:
        batch_x = test_X[i : min(i + batch_size, test_X.shape[0])]
        batch_y = test_Y[i : min(i + batch_size, test_X.shape[0])]
        batch_memory = np.zeros(([d_model, 0]))
        batch_x_expand = np.expand_dims(batch_x,axis = 1)
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
                model.memory: batch_memory
            },
        )
        test_loss += cost
        test_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)

    train_loss /= len(train_X) / batch_size
    train_acc /= len(train_X) / batch_size
    test_loss /= len(test_X) / batch_size
    test_acc /= len(test_X) / batch_size

    if test_acc > CURRENT_ACC:
        print(
            'epoch: %d, pass acc: %f, current acc: %f'
            % (EPOCH, CURRENT_ACC, test_acc)
        )
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1

    print('time taken:', time.time() - lasttime)
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (EPOCH, train_loss, train_acc, test_loss, test_acc)
    )
    EPOCH += 1

train minibatch loop:   1%|          | 2/267 [00:00<00:23, 11.15it/s, accuracy=0.781, cost=0.63]

epoch: 0, pass acc: 0.000000, current acc: 0.501708
time taken: 26.53965997695923
epoch: 0, training loss: 0.743792, training acc: 0.498879, valid loss: 0.717732, valid acc: 0.501708



train minibatch loop:   1%|          | 2/267 [00:00<00:23, 11.12it/s, accuracy=0.75, cost=0.514]

time taken: 26.217947244644165
epoch: 1, training loss: 0.656768, training acc: 0.613609, valid loss: 0.743042, valid acc: 0.493738



train minibatch loop:   1%|          | 2/267 [00:00<00:23, 11.13it/s, accuracy=0.875, cost=0.354]

...

time taken: 26.2944438457489
epoch: 3, training loss: 0.541744, training acc: 0.722373, valid loss: 1.016386, valid acc: 0.501261



train minibatch loop:   1%|          | 2/267 [00:00<00:23, 11.06it/s, accuracy=0.781, cost=0.667]

time taken: 26.34887981414795
epoch: 4, training loss: 0.423682, training acc: 0.800397, valid loss: 1.273735, valid acc: 0.487398



test minibatch loop: 100%|██████████| 67/67 [00:02<00:00, 31.43it/s, accuracy=0.333, cost=1.77] 

time taken: 26.30614972114563
epoch: 5, training loss: 0.267241, training acc: 0.886077, valid loss: 1.177130, valid acc: 0.485545

break epoch:6



In [12]:
real_Y, predict_Y = [], []

pbar = tqdm(
    range(0, len(test_X), batch_size), desc = 'validation minibatch loop'
)
for i in pbar:
    batch_x = test_X[i : min(i + batch_size, test_X.shape[0])]
    batch_y = test_Y[i : min(i + batch_size, test_X.shape[0])]
    predict_Y += np.argmax(
        sess.run(
            model.logits, feed_dict = {model.X: batch_x, model.Y: batch_y}
        ),
        1,
    ).tolist()
    real_Y += batch_y

validation minibatch loop: 100%|██████████| 67/67 [00:02<00:00, 32.24it/s]


In [13]:
print(metrics.classification_report(real_Y, predict_Y, target_names = trainset.target_names))

             precision    recall  f1-score   support

   negative       0.47      0.34      0.39      1061
   positive       0.49      0.63      0.55      1072

avg / total       0.48      0.48      0.47      2133

